# Segmenting and Clustering Neighborhoods in Toronto

### For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

# **_Question 1_**
___
### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [149]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [150]:
# import beautifulsoup
from bs4 import BeautifulSoup

# use older URL in order to complete all portions of assignment
wiki_url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=862527922"
source = requests.get(wiki_url).text

# use beautifulsoup to scrape
soup = BeautifulSoup(source, 'xml')
table = soup.find('table',{'class':'wikitable sortable'})

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [151]:
# extract table from site and clean up columns
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    td=[]
    for t in row.find_all('td'):
        td.append(t.text.strip())
    data.append(td)
df = pd.DataFrame(data, columns=['Postal Code', 'Borough', 'Neighborhood'])
df.head(10)

,Postal Code,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [152]:
# filter out rows without borough or Not assigned
df = df[~df['Borough'].isnull()]
df.drop(df[df.Borough == 'Not assigned'].index, inplace = True)
df.reset_index(drop = True, inplace = True)

df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [153]:
# add commas and join neighborhoods
df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()
df['Neighborhood'].replace('Not assigned', df['Borough'], inplace = True)

df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [154]:
df.shape

(103, 3)

# **_Question 2_**
___
### Use the Geocoder package or the csv file to get the latitude and the longitude coordinates of each neighborhood and recreate the dataframe in the example

In [155]:
# use csv file containing coordinates

coord_df=pd.read_csv('http://cocl.us/Geospatial_data')
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [156]:
# merge tables and order columns

coord_merged = pd.merge(coord_df, df, on='Postal Code')
coord_df = coord_merged[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
coord_df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# **_Question 3_**
---

### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

### Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. 

In [157]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [158]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(coord_df['Latitude'], coord_df['Longitude'], coord_df['Borough'], coord_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Work with only the Scarborough borough and then replicate the same analysis we did to the New York City data in the lab.

In [159]:
# narrow down data to only work within Scarborough borough
scar_data = coord_df[coord_df['Borough'] == 'Scarborough'].reset_index(drop=True)
scar_data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [160]:
# get coordinates of Scarborough

address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


### As we did with all of Toronto, let's visualize Scarborough and the neighborhoods in it.

In [161]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scar_data['Latitude'], scar_data['Longitude'], scar_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Utilize the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [162]:
CLIENT_ID = 'YMUL3HU2WHW12ZPQTAS4VIGTD2S0FYZLBRPIT5PN4ZX4ZOWS' # your Foursquare ID
CLIENT_SECRET = 'CECEBTYIOD214WSZKIFAI5S0UVNKL333OXHJOJPFS2S0BRN1' # your Foursquare Secret
VERSION = '20200416' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YMUL3HU2WHW12ZPQTAS4VIGTD2S0FYZLBRPIT5PN4ZX4ZOWS
CLIENT_SECRET:CECEBTYIOD214WSZKIFAI5S0UVNKL333OXHJOJPFS2S0BRN1


### Explore first neighborhood in dataframe

In [163]:
scar_data.loc[0, 'Neighborhood']

'Rouge, Malvern'

### Get the neighborhood's latitude and longitude values.

In [164]:
neighborhood_latitude = scar_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scar_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scar_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


### Get the top 50 venues that are in Rogue, Malvern within a radius of 1000 meters.

In [165]:
LIMIT = 50 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YMUL3HU2WHW12ZPQTAS4VIGTD2S0FYZLBRPIT5PN4ZX4ZOWS&client_secret=CECEBTYIOD214WSZKIFAI5S0UVNKL333OXHJOJPFS2S0BRN1&v=20200416&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=50'

In [166]:
#Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9cb8711a4b0a001bddb0cd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '579a91b3498e9bd833afa78a',
       'name': "Wendy's",
       'location': {'address': '8129 Sheppard Avenue',
        'lat': 43.8020084,
        'lng': -79.1980797,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.8020084,
          'ln

In [167]:
# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the json and structure it into a pandas dataframe.

In [168]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.802008,-79.198080
1,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
2,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
3,Harvey's,Restaurant,43.800020,-79.198307
4,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
5,Tim Hortons,Coffee Shop,43.802000,-79.198169
6,Lee Valley,Hobby Shop,43.803161,-79.199681
7,Bus Stop: 85 & 116,Bus Station,43.802198,-79.199389
8,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296
9,Rouge Park - Woodland Trail,Trail,43.801782,-79.200427


In [169]:
# check the number of neighborhoods returned by Foursquare

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


## Explore Neighborhoods in Scarborough

In [173]:
# create a function to repeat the same process to all the neighborhoods in Scarborough

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Rrun the above function on each neighborhood and create a new dataframe called *scar_venues*

In [174]:
scar_venues = getNearbyVenues(names=scar_data['Neighborhood'],
                                   latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [175]:
# check size of dataframe

print(scar_venues.shape)
scar_venues.head()

(412, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
3,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store


In [176]:
# check how many venues were returned for each neighborhood

scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,50,50,50,50,50,50
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",31,31,31,31,31,31
"Birch Cliff, Cliffside West",12,12,12,12,12,12
Cedarbrae,31,31,31,31,31,31
"Clairlea, Golden Mile, Oakridge",29,29,29,29,29,29
"Clarks Corners, Sullivan, Tam O'Shanter",40,40,40,40,40,40
"Cliffcrest, Cliffside, Scarborough Village West",12,12,12,12,12,12
"Dorset Park, Scarborough Town Centre, Wexford Heights",45,45,45,45,45,45
"East Birchmount Park, Ionview, Kennedy Park",30,30,30,30,30,30


In [177]:
# find out how many unique categories can be curated from the returned venues

print('There are {} uniques categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 112 uniques categories.


### Analyze each neighborhood

In [178]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Board Shop,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,German Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Hardware Store,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [179]:
# get new dataframe size

scar_onehot.shape

(412, 113)

In [180]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

,Neighborhood,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Board Shop,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,German Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Hardware Store,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.02,0.020000,0.000000,0.000000,0.000000,0.020000,0.040000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020,0.040000,0.160000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.020000,0.000000,0.000000,0.02,0.000000,0.000000,0.02,0.020000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.02,0.000000,0.000000,0.020000,0.000000,0.020000,0.040000,0.0,0.000000,0.02,0.02,0.02,0.000000,0.000000,0.020000,0.040000,0.020000,0.000000,0.000000,0.060000,0.020000,0.000000,0.000000,0.02,0.020000,0.02,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.020000,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.00,0.000000,0.000000,0.000000,0.064516,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.032258,0.161290,0.032258,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.032258,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.032258,0.00,0.000000,0.000000,0.00,0.032258,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.00,0.000000,0.000000,0.064516,0.000000,0.064516,0.064516,0.0,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000,0.000000,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [181]:
# confirm new size

scar_grouped.shape

(16, 113)

In [182]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.16
1         Shopping Mall  0.06
2           Coffee Shop  0.04
3           Pizza Place  0.04
4  Caribbean Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0    Chinese Restaurant  0.16
1                  Park  0.06
2  Fast Food Restaurant  0.06
3             BBQ Joint  0.06
4                Bakery  0.06


----Birch Cliff, Cliffside West----
                   venue  freq
0                  Diner  0.08
1      Convenience Store  0.08
2                    Gym  0.08
3               Gym Pool  0.08
4  General Entertainment  0.08


----Cedarbrae----
               venue  freq
0        Coffee Shop  0.10
1             Bakery  0.10
2  Indian Restaurant  0.06
3        Gas Station  0.06
4               Bank  0.06


----Clairlea, Golden Mile, Oakridge----
               venue  freq
0       Intersection  0.10
1        Coffee Shop  0.07
2  Convenience Store  0.07

### Put the above into a *pandas* dataframe

In [183]:
# sort venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [184]:
# create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Sandwich Place,Coffee Shop,Caribbean Restaurant,Bakery,Pizza Place,Indian Restaurant,Bubble Tea Shop,Print Shop
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Park,Pharmacy,Pizza Place,BBQ Joint,Bakery,Fast Food Restaurant,Shopping Mall,Shop & Service,Noodle House
2,"Birch Cliff, Cliffside West",Ice Cream Shop,Skating Rink,Gym Pool,Gym,General Entertainment,Diner,Park,Convenience Store,Restaurant,Café
3,Cedarbrae,Bakery,Coffee Shop,Gas Station,Indian Restaurant,Bank,Yoga Studio,Board Shop,Caribbean Restaurant,Fried Chicken Joint,Bus Line
4,"Clairlea, Golden Mile, Oakridge",Intersection,Coffee Shop,Diner,Convenience Store,Bus Line,Bakery,Restaurant,Pub,Pizza Place,Park


## Cluster neighborhoods

In [190]:
# Run k-means to cluster the neighborhood into 4 clusters.

# set number of clusters
kclusters = 4

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 0], dtype=int32)

In [191]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = scar_data

# merge to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scar_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Coffee Shop,Fast Food Restaurant,Trail,Greek Restaurant,Chinese Restaurant,Caribbean Restaurant,Restaurant,Sandwich Place,Bus Station,Supermarket
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Italian Restaurant,Park,Burger Joint,Playground,Breakfast Spot,Yoga Studio,Furniture / Home Store,Diner,Discount Store,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Pizza Place,Coffee Shop,Grocery Store,Fast Food Restaurant,Bank,Sports Bar,Pharmacy,Discount Store,Electronics Store,Rental Car Location
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Park,Coffee Shop,Pharmacy,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Mobile Phone Shop,Yoga Studio,Fried Chicken Joint,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Bakery,Coffee Shop,Gas Station,Indian Restaurant,Bank,Yoga Studio,Board Shop,Caribbean Restaurant,Fried Chicken Joint,Bus Line


### Visualize resulting clusters

In [215]:
scar_merged.dropna()
scar_merged['Cluster Labels'] = scar_merged['Cluster Labels'].fillna(0).astype(int)

In [216]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

In [217]:
# cluster 1
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0,Park,Coffee Shop,Pharmacy,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Mobile Phone Shop,Yoga Studio,Fried Chicken Joint,Discount Store
16,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
# cluster 2
scar_merged.loc[scar_merged['Cluster Labels'] == 1, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Coffee Shop,Fast Food Restaurant,Trail,Greek Restaurant,Chinese Restaurant,Caribbean Restaurant,Restaurant,Sandwich Place,Bus Station,Supermarket
2,Scarborough,1,Pizza Place,Coffee Shop,Grocery Store,Fast Food Restaurant,Bank,Sports Bar,Pharmacy,Discount Store,Electronics Store,Rental Car Location
4,Scarborough,1,Bakery,Coffee Shop,Gas Station,Indian Restaurant,Bank,Yoga Studio,Board Shop,Caribbean Restaurant,Fried Chicken Joint,Bus Line
5,Scarborough,1,Ice Cream Shop,Fast Food Restaurant,Pizza Place,Women's Store,Convenience Store,Train Station,Coffee Shop,Japanese Restaurant,Sandwich Place,Grocery Store
6,Scarborough,1,Discount Store,Coffee Shop,Chinese Restaurant,Grocery Store,Fast Food Restaurant,Light Rail Station,Department Store,Pharmacy,Pizza Place,Hockey Arena
7,Scarborough,1,Intersection,Coffee Shop,Diner,Convenience Store,Bus Line,Bakery,Restaurant,Pub,Pizza Place,Park
9,Scarborough,1,Ice Cream Shop,Skating Rink,Gym Pool,Gym,General Entertainment,Diner,Park,Convenience Store,Restaurant,Café
10,Scarborough,1,Restaurant,Furniture / Home Store,Coffee Shop,Electronics Store,Indian Restaurant,Asian Restaurant,Chinese Restaurant,Pharmacy,Bakery,Fast Food Restaurant
11,Scarborough,1,Middle Eastern Restaurant,Pizza Place,Grocery Store,Burger Joint,Intersection,Soccer Field,Pharmacy,Coffee Shop,Café,Bus Station
12,Scarborough,1,Chinese Restaurant,Shopping Mall,Sandwich Place,Coffee Shop,Caribbean Restaurant,Bakery,Pizza Place,Indian Restaurant,Bubble Tea Shop,Print Shop


In [219]:
# cluster 3
scar_merged.loc[scar_merged['Cluster Labels'] == 2, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Italian Restaurant,Park,Burger Joint,Playground,Breakfast Spot,Yoga Studio,Furniture / Home Store,Diner,Discount Store,Electronics Store


In [220]:
# cluster 4
scar_merged.loc[scar_merged['Cluster Labels'] == 3, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,3,Pizza Place,Ice Cream Shop,Beach,Park,Cajun / Creole Restaurant,Burger Joint,Sports Bar,Hardware Store,Food & Drink Shop,Diner
